# 1.TIPO DE CAMBIO

In [ ]:
##DATO EXTRAIDO TIPO DE CAMBIO

In [2]:
import requests
import pandas as pd

# API abierta de tipo de cambio actual USD -> PEN
url = "https://open.er-api.com/v6/latest/USD" ### API UTILIZADA

headers = {
    "User-Agent": "proyecto-forecast-lizeth/1.0"
}

response = requests.get(url,headers=headers)
data = response.json()

# Extraer tipo de cambio
tipo_cambio = data['rates']['PEN']
fecha = data['time_last_update_utc']
# Crear DataFrame
df_tc = pd.DataFrame([{
    'fecha': pd.to_datetime(fecha), ##DATOS EXTRAIDO
    'tipo_cambio': tipo_cambio ##DATO EXTRAIDO
}])

print(df_tc)



                      fecha  tipo_cambio
0 2025-04-04 00:02:32+00:00     3.666956


**JUSTIFICACION**

En una estrategia de precios, el tipo de cambio influye en el costo de productos importados. Si el dato está desactualizado o tiene valores extremos, puede generar decisiones erróneas sobre precios o márgenes. Por eso es clave validar su confiabilidad.

In [ ]:
##INDICADORES DE CALIDAD##

In [19]:
#1.Actualización:
##VER LA ÚLTIMA FECHA
##La API devuelve la fecha de la última actualización (`time_last_update_utc`) para asegurar que los datos están al día.
fecha = data['time_last_update_utc']
print(fecha)

Fri, 04 Apr 2025 00:02:32 +0000


In [15]:
##2. COMPLETITUD: VERIFICA QUE EL VALOR DE TIPO DE CAMBIO NO ES NULO
##PARA VER SI HAY NULOS
print(df_tc.isnull().sum())

fecha          0
tipo_cambio    0
dtype: int64


In [16]:
#3. RANGO LÓGICO : COMPROBAR QUE EL TIPO DE CAMBIO ESTÉ DENTRO DE UN VALOR REALISTA
##PARA VER SI EL RANGO ES LÓGICO
print(df_tc['tipo_cambio'].describe())

count    1.000000
mean     3.666956
std           NaN
min      3.666956
25%      3.666956
50%      3.666956
75%      3.666956
max      3.666956
Name: tipo_cambio, dtype: float64


In [ ]:
###En una estrategia de precios, el tipo de cambio influye en el costo de productos importados. Si el dato está desactualizado o tiene valores extremos, puede generar decisiones erróneas sobre precios o márgenes. Por eso es clave validar su confiabilidad.

# 2.TEMPERATURA

In [ ]:
#DATO EXTRAIDO = TEMPERATURA HORARIA ESTIMADA EN LIMA DURANTE LAS PRÓXIMAS 24 HORAS.

In [1]:
import requests
import pandas as pd

# Coordenadas de Lima
lat = -12.0464
lon = -77.0428

# API de clima de MET Norway
url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"

# MET Norway requiere que definas un "User-Agent"
headers = {
    "User-Agent": "proyecto-forecast-lizeth/1.0"
}

response = requests.get(url, headers=headers)
data = response.json()

# Extraer temperatura de los primeros registros
forecast = data['properties']['timeseries']

temperaturas = []
for punto in forecast[:24]:  # Tomamos primeras 24 horas
    fecha = punto['time']
    temp = punto['data']['instant']['details']['air_temperature']
    temperaturas.append({'fecha': fecha, 'temperatura': temp})

df_temp = pd.DataFrame(temperaturas)
df_temp['fecha'] = pd.to_datetime(df_temp['fecha'])

print(df_temp.head())


                      fecha  temperatura
0 2025-04-04 17:00:00+00:00         27.3
1 2025-04-04 18:00:00+00:00         26.9
2 2025-04-04 19:00:00+00:00         27.0
3 2025-04-04 20:00:00+00:00         27.1
4 2025-04-04 21:00:00+00:00         26.2


In [ ]:
##INDICADORES DE CALIDAD APLICADOS


In [3]:
#1. COMPLETITUD
#VERIFICANDO QUE NO HAYA REGISTROS NULOS EN LA COLUMNA 'temperatura'
print(df_temp['temperatura'].isnull().sum())

0


In [4]:
#2.RANGO LÓGICO
#Se comprobó que los valores estén dentro del rango esperado para Lima (entre 10°C y 40°C).
print(df_temp['temperatura'].describe())

count    24.000000
mean     23.362500
std       2.750939
min      20.100000
25%      20.875000
50%      22.350000
75%      26.375000
max      27.300000
Name: temperatura, dtype: float64


In [5]:
#3.FORMATO DE FECHA
#Las fechas fueron convertidas correctamente al formato `datetime` para garantizar su uso en series temporales.
print(df_temp['fecha'].min(), '→', df_temp['fecha'].max())


2025-04-04 17:00:00+00:00 → 2025-04-05 16:00:00+00:00


**JUSTIFICACIÓN**

La temperatura puede afectar la demanda de ciertos productos, por ejemplo bebidas frías, ropa ligera, o decisiones logísticas. Datos incompletos o fuera de rango podrían afectar la precisión de un modelo de forecast o de decisiones operativas basadas en clima.

# 3.NOTICIAS

In [14]:
import requests
import pandas as pd

api_key = #insertar tu api_key te da 200 creditos al registrarte obtener en la pagina https://newsdata.io/   #

# Endpoint demo (limitado pero funcional)
url = f"https://newsdata.io/api/1/news?apikey={api_key}&q=escasez+precios&language=es&country=pe"

headers = {
    "User-Agent": "forecast-noticias-tino/1.0"
}

response = requests.get(url, headers=headers)
data = response.json()
print(data)

# Extraer noticias
noticias = []
for item in data['results']:
    noticias.append({
        'titulo': item['title'],
        'fuente': item.get('source_id','Desconocida'),
        'fecha': item['pubDate'],
        'descripcion': item['description'],
        'url': item['link']
    })


# Convertir a DataFrame
df_news = pd.DataFrame(noticias)
df_news['fecha'] = pd.to_datetime(df_news['fecha'])

print(df_news.head())


{'status': 'success', 'totalResults': 9, 'results': [{'article_id': '9ba87d2583bd66cfc69ae4783c3c196e', 'title': 'Malas noticias para inmigrantes en EEUU: un estado implementa nuevas medidas para arrestar indocumentados por delitos menores', 'link': 'https://larepublica.pe/mundo/2025/04/03/malas-noticias-para-inmigrantes-en-estados-unidos-un-estado-implementa-nuevas-medidas-para-arrestar-indocumentados-por-delitos-menores-lrtmus-91392', 'keywords': ['mundo'], 'creator': None, 'video_url': None, 'description': 'En Estados Unidos, un estado implementa nuevas directrices que permiten la detención de inmigrantes indocumentados por infracciones menores, como conducir sin licencia, generando controversia.', 'content': 'ONLY AVAILABLE IN PAID PLANS', 'pubDate': '2025-04-03 23:57:02', 'pubDateTZ': 'UTC', 'image_url': 'https://imgmedia.larepublica.pe/1200x630/larepublica/original/2025/04/03/67ef1c8a0d59007ae302b1a6.jpg', 'source_id': 'larepublica', 'source_priority': 7044, 'source_name': 'La R

In [ ]:
##INDICADORES DE CALIDAD APLICADOS


In [15]:
##1. COMPLETITUD ##
#Validé que campos como `titulo`, `fuente` y `fecha` estén completos#
print(df_news.isnull().sum())

titulo         0
fuente         0
fecha          0
descripcion    0
url            0
dtype: int64


In [16]:
##2.NO DUPLICIDAD##
#Se comprobó que no haya títulos repetidos.
print(df_news.duplicated(subset='titulo').sum())

0


In [17]:
##3.RELEVANCIA TEMÁTICA##
#Se busca noticias relacionadas según el tema
palabras_clave = ['escasez', 'precio', 'precios', 'demanda', 'inventario','temporada']
#se convierte en minúsuclas para una correcta comparación (los títulos)
df_news['texto_completo'] = df_news['titulo'].str.lower() + " " + df_news['descripcion'].str.lower()

# Crear una columna para contar cuántas veces aparece alguna palabra clave
df_news['relevante'] = df_news['texto_completo'].apply(
    lambda texto: any(palabra in texto for palabra in palabras_clave)
)

# Filtrar noticias relevantes
df_relevantes = df_news[df_news['relevante'] == True]

# Mostrar
print("🔎 Noticias relevantes según términos clave:")
print(df_relevantes[['titulo', 'fecha', 'fuente']])


🔎 Noticias relevantes según términos clave:
                                              titulo               fecha  \
3  Aranceles en Estados Unidos: consumidores se a... 2025-04-03 13:52:34   
4  Precio de los huevos en EE. UU.: ¿cuánto cues... 2025-04-03 01:25:23   
8  Presidente de Boticas y Farmacias del Perú al... 2025-04-02 18:39:24   

        fuente  
3  larepublica  
4  larepublica  
8  larepublica  


JUSTIFICACIÓN

Las noticias económicas permiten identificar eventos que afectan la oferta o demanda, como desabastecimientos o alzas de precios. Son un insumo clave para una estrategia de precios predictiva. Por lo que buscar por palabras claves facilita la busqueda de noticias (datos) relacionados que podrían afectar la previsión de inventarios